어려우니 뭐를 하는지를 먼저 이해하기

# TFRecord
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- Tensorflow에서 제공하는 데이터셋 파일저장 방식.
    - 데이터 양이 많을 경우 이를 Binary로 **Seralization(직렬화)하여 하나의 파일**로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer 형태로 Serialization을 수행해서 저장할 수 있는 TFRecords 파일 포맷 형태를 지원한다.
        - 직렬화: 어떤 데이터를 지정한 포맷으로 파일로 저장하는 것
        - Protocol Buffer: 자체 파일 포맷. 딕셔너리 형태
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecords 파일에 저장할 수 있다.
- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.
- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세 가지 타입을 지원한다. value의 타입에 따라
        - tf.train.BytesList – string, byte 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환
- tf.tran.Example의 형태 - k:v. k는 문자열
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```

In [1]:
import tensorflow as tf

In [2]:
# tf.train.Feature 객체들을 생성하는 (기본타입의 값들을 Feature로 변환하는) 함수 구현
def _bytes_feature(value):
    """
    value로 string, bytes 를 받아서 BytesList로 변환하는 함수
    언더바: 외부에서 호출하지 못하도록
    """
    # value가 Tensor(텐서플로우의 배열 타입)타입인 경우 ndarray로 변환 => binary 파일
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """
    float타입의 value를 받아서 FloatList로 변환하는 함수
    """
    # float은 그대로 값이 넘어오기 때문에 타입변환 필요없음
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """
    int, uint, bool 타입의 value를 받아서 Int64List로 변환하는 함수
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
v = _int64_feature(20)
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
int64_list {
  value: 20
}



In [4]:
v = _float_feature(56.7) # 타입은 똑같이 Feature. float_list
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
float_list {
  value: 56.7
}



In [5]:
v = _int64_feature(True) # bool: 0,1 정수로 바뀌어서 들어감
print(v)

int64_list {
  value: 1
}



In [6]:
v = _bytes_feature(b'hello') # 문자열, bytes => bytes 타입으로 전달
print(v)                     # 문자열 앞에 접두어 b를 붙여줘야 byte로 인식

bytes_list {
  value: "hello"
}



In [7]:
s = '홍길동' # 일반 string일 경우 => bytes 타입으로 변환한 뒤에 BytesList로 변환
_bytes_feature(s.encode('utf-8')) # encode: bytes 타입으로 바꿔줌. 영문은 그대로 보여지고, 한글은 풀어서 보여짐

bytes_list {
  value: "\355\231\215\352\270\270\353\217\231"
}

In [8]:
# binary 파일 (이미지) - 일반적으로 binary라고 하고 파이썬에서는 bytes타입을 사용
with open('lion.jpg', 'rb') as f:
    img = f.read()
type(img)

bytes

In [9]:
v = _bytes_feature(img)
print(type(v))
print(v)

<class 'tensorflow.core.example.feature_pb2.Feature'>
bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\001\000H\000H\000\000\377\342\014XICC_PROFILE\000\001\001\000\000\014HLino\002\020\000\000mntrRGB XYZ \007\316\000\002\000\t\000\006\0001\000\000acspMSFT\000\000\000\000IEC sRGB\000\000\000\000\000\000\000\000\000\000\000\000\000\000\366\326\000\001\000\000\000\000\323-HP  \000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\021cprt\000\000\001P\000\000\0003desc\000\000\001\204\000\000\000lwtpt\000\000\001\360\000\000\000\024bkpt\000\000\002\004\000\000\000\024rXYZ\000\000\002\030\000\000\000\024gXYZ\000\000\002,\000\000\000\024bXYZ\000\000\002@\000\000\000\024dmnd\000\000\002T\000\000\000pdmdd\000\000\002\304\000\000\000\210vued\000\000\003L\000\000\000\206view\000\000\003\324\000\000\000$lumi\000\000\003\370\000\000\000\024meas\000\00

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력하기 전에 변환해야 한다.

In [10]:
feature = _float_feature(30.2)
print(feature)

float_list {
  value: 30.2
}



In [12]:
v = feature.SerializeToString() # 출력형태로 변환. String => bytes개념. 문자열이 아니라
type(v)

bytes

In [13]:
print(v)

b'\x12\x06\n\x04\x9a\x99\xf1A'


# TFRecord 저장 예제

### tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.


### TFRecord로 저장할 여러 타입의, 가상의 toy dataset을 생성

In [37]:
import numpy as np
N_DATA = 1000 # dataset의 데이터 개수

# bool 1000개 생성
feature0 = np.random.choice([False, True], N_DATA) # 가상의 dataset
# 정수 1000개
feature1 = np.random.randint(0,5,N_DATA)
# string 1000
str_list = np.array([b'lion', b'tiger', b'cat', b'dog', b'bear'])
feature2 = str_list[feature1]
# float 1000
feature3 = np.random.randn(N_DATA)
feature0.shape, feature1.shape, feature2.shape, feature3.shape

((1000,), (1000,), (1000,), (1000,))

In [38]:
def serialize_example(feature0, feature1, feature2, feature3):
    """
    '하나의 데이터'를 속성(Feature)값들을 받아서 Exmaple을 생성한 뒤 그 Exmaple을 출력 가능한 bytes로 만들어 반환(SerializeToString()이용하여 직렬화한 후 반환)
    feature0: bool, feature1: int, feature2: bytes(string), feature3: float
    """
    # feature들을 dictionary로 생성
    feature = {
        'feature0':_int64_feature(feature0),
        'feature1':_int64_feature(feature1),
        'feature2':_bytes_feature(feature2),
        'feature3':_float_feature(feature3), # 딕셔너리,리스트 만들 때 마지막에 쉼표 넣는 습관
    }
    # feature들을 가진 tf.train.Example 객체를 생성
    example = tf.train.Example(features=tf.train.Features(feature=feature))
#     print(type(example))
    
    # Example을 TFRecord에 저장하기 위한 형태인 bytes로 변환 -> SerializeToString() 이용
    return example.SerializeToString()

### 출력 처리
- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [39]:
import os
# TFRecord 파일 저장할 디렉토리를 생성
if not os.path.isdir('tfrecord'):
    os.mkdir('tfrecord')

In [40]:
# TFRecordWriter 객체를 생성 -> TFRecord파일로 직렬화된 Example을 출력하는 메소드 제공
tfrecord_file_path = './tfrecord/data.tfr' # 확장자: tfr, record, tfrecord
tfrecord_writer = tf.io.TFRecordWriter(tfrecord_file_path) # tfrecord_file_path경로에 data.tfr파일로 저장

In [41]:
for data in zip(feature0, feature1, feature2, feature3):
    sv = serialize_example(bool(data[0]), data[1], data[2], data[3]) # bool을 넘파이배열로 하면서 변환이 왜 안됨
    tfrecord_writer.write(sv)
    
tfrecord_writer.close() # 출력 Stream 연결 닫기

# 위의 모든 데이터들이 필요하면 data.tfr 하나만 가져가면 되는 것임. 이 파일만 있으면 속의 이미지 파일들이 내 로컬 컴퓨터에 없어도 사용, 학습 가능
# 직렬화 => 출력할 수 있는 bytes로 만들어주는 것

# TFRecord파일 읽기 및 역직렬화(Deserialize)
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.
- 직렬화 -> 출력 개념. 역직렬화 -> 읽기 개념

In [52]:
def _parse_function(tfrecord_serialized):
    """
    serialize(직렬화-bytes)된 Example 데이터 하나를 받아서 역직렬화한 뒤에 반환하는 함수
    [매개변수]
        tfrecord_serialized: 직렬화된 Example
    [반환값]
        Example구성 Feature들. (feature0, 1, 2, 3)
    """
    # 역직렬화해서 읽어온 Feature들을 저장할 Feature들을 dictionary로 구성
    # 이름: 저장할 때 사용한 name, value: 읽어온 Feature를 저장할 빈 Feature (역직렬화할 값의 타입을 선언한다)
    feature = {
        'feature0':tf.io.FixedLenFeature([], tf.int64),
        'feature1':tf.io.FixedLenFeature([], tf.int64),
        'feature2':tf.io.FixedLenFeature([], tf.string),
        'feature3':tf.io.FixedLenFeature([], tf.float32),
    }
    parsed_features = tf.io.parse_single_exmaple(tfrecord_serialized, feature)
    # tf.cast(값, dtype): 값의 데이터 타입을 지정한 dtype으로 변환
    feature0 = tf.cast(parsed_features['feature0'], tf.bool) # int64 -> 원래 타입으로 형변환해줘야 함
    feature1 = tf.cast(parsed_features['feature1'], tf.int64)
    feature2 = tf.cast(parsed_features['feature2'], tf.string)
    feature3 = tf.cast(parsed_features['feature3'], tf.float32)
    return feature0, feature1, feature2, feature3

In [53]:
# tfrecord 파일 읽기
dataset = tf.data.TFRecordDataset(tfrecord_file_path).map(_parse_function) 
# 어느 경로에서 읽어올지 지정만 해주는 것. 실제 읽어오는 건 모델에 데이터 feeding해줄 때
# map을 써서 실제로 추출 -> example이 들어가서 (f0,f1,,f2,f3) 튜플이 반환되도록

AttributeError: in converted code:

    <ipython-input-33-ee826edffbba>:17 _parse_function  *
        parsed_features = tf.io.parse_single_exmaple(tfrecord_serialized, feature)

    AttributeError: module 'tensorflow_core._api.v2.io' has no attribute 'parse_single_exmaple'


data모음에는 이 두 가지 함수 무조건 있음
- map(): 함수를 처리한 후 그 결과값을 반환
- filter(): bool형태. 필터해서 해당되는 것들 걸러내는 것

In [54]:
# dataset에서 3개의 데이터만 조회 -> 이 작업은 fit할 때 내부에서 일어남
for data in dataset.take(3):
#     print(type(data))
    print(data[0].numpy(), data[1].numpy(), data[2].numpy(), data[3].numpy())

NameError: name 'dataset' is not defined